In [1]:
# Clone PyFM
!git clone https://github.com/RobinMagnet/pyFM.git pyfm_repo
!mv pyfm_repo/pyFM .
!rm -rf pyfm_repo

Cloning into 'pyfm_repo'...
remote: Enumerating objects: 512, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 512 (delta 59), reused 56 (delta 50), pack-reused 436
Receiving objects: 100% (512/512), 942.38 KiB | 8.89 MiB/s, done.
Resolving deltas: 100% (324/324), done.
mv: rename pyfm_repo/pyFM to ./pyFM: Directory not empty



# 1 - Imports and defining functions

In [2]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

# 2 - Computing the functional map

**Loading data**

In [16]:
num=6
mesh2 = TriMesh('data/dataset/BlueToy02/push_toy_1_70000.obj')
mesh1 = TriMesh("data/dataset/BlueToy02/push_toy_"+str(num)+"_70000.obj")
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

double_plot(mesh1,mesh2)

Mesh 1 : 36903 vertices, 69999 faces
Mesh 2 : 35002 vertices, 70000 faces


In [4]:
from scipy.spatial import distance
import numpy as np
rand_idx=np.random.permutation(mesh1.vertlist.shape[0])[:20]
print(rand_idx)
euc_dist=distance.cdist(mesh1.vertlist[rand_idx],mesh2.vertlist)

[ 8465  5189 18285 16762  3130 16057 34914 20767  4060 24367 22302 11130
 36760 16432  4812  3957 36493 14684  4821  1179]


In [5]:
temp=np.argmin(euc_dist, axis=1)

In [6]:
# Manually change the landmark file name
with open("data/blue_toy_1-"+str(num)+"_landmarks_deformed_and_non.txt", 'w+') as fp:
    
    for i in range(temp.shape[0]):
        fp.write(str(rand_idx[i])+" "+str(temp[i])+"\n")

with open("data/blue_toy_1-"+str(num)+"_landmarks_non_and_deformed.txt", 'w+') as fp:
    
    for i in range(temp.shape[0]):
        fp.write(str(temp[i])+" "+str(rand_idx[i])+"\n")


**Computing descriptors**

In [8]:
process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt("data/blue_toy_1-"+str(num)+"_landmarks_deformed_and_non.txt",dtype=int),  # loading 10 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}

model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);


Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 12.55 s
Computing 200 eigenvectors
	Done in 14.00 s

Computing descriptors
	Normalizing descriptors

	420 out of 2100 possible descriptors kept


**Fitting the model**

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\Ss}{\mathcal{S}}$
$\newcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\newcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

In pyFM, we always consider functional maps $\*C:\Ss_1\to\Ss_2$ and pointwise maps $T:\Ss_2\to\Ss_1$ going in opposite directions, with $\*C$ always going from shape 1 to shape 2 !

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} w_{descr}\|\*C\*A - \*B\|^2 + w_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + w_{\text{d- comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + w_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ [multipliative operators](http://www.lix.polytechnique.fr/~maks/papers/fundescEG17.pdf) associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ [orientation preserving operators](https://arxiv.org/abs/1806.04455) associated to the $i$-th descriptors

In [9]:
fit_params = {
    'w_descr': 1e0,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
}



model.fit(**fit_params, verbose=True)

Computing commutativity operators
	Scaling LBO commutativity weight by 1.5e-11

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 420 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 18, nit : 15, warnflag : 0
	Done in 0.80 seconds


**Visualizing the associated point to point map**

In [10]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

# 3 - Refining the Functional Map
```model.FM``` returns the current state of functional map. One can change which one is returned by using ```model.change_FM_type(FM_type)```, as one can see below. 

**ICP**

In [11]:
model.icp_refine(verbose=True)
p2p_21_icp = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/10 [00:00<?, ?it/s]

**Zoomout**

In [12]:
model.change_FM_type('classic') # We refine the first computed map, not the icp-refined one
model.zoomout_refine(nit=15, step = 1, verbose=True)
print(model.FM.shape)
p2p_21_zo = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/15 [00:00<?, ?it/s]

(50, 50)


In [13]:
print(mesh1.vertlist[p2p_21_icp].shape,mesh2.vertlist.shape)
np.save("Non-deformed-to-Deformed Correspondences/BlueToy02/deformed_"+str(num)+"_correspondences_icp.npy",mesh1.vertlist[p2p_21_icp])
np.save("Non-deformed-to-Deformed Correspondences/BlueToy02/non_deformed_"+str(num)+"_correspondences_icp.npy",mesh2.vertlist )

(35002, 3) (35002, 3)


In [14]:
print(mesh1.vertlist[p2p_21_zo].shape,mesh2.vertlist.shape)
np.save("Non-deformed-to-Deformed Correspondences/BlueToy02/deformed_"+str(num)+"_correspondences_zoomed.npy",mesh1.vertlist[p2p_21_zo])
np.save("Non-deformed-to-Deformed Correspondences/BlueToy02/non_deformed_"+str(num)+"_correspondences_zoomed.npy",mesh2.vertlist )

(35002, 3) (35002, 3)


### Finding bijective Correspondences

In [3]:
num=4
a=np.load("Deformed-to-Non-deformed Correspondences/BlueToy02/"+"deformed_"+str(num)+"_correspondences_icp.npy")
b=np.load("Non-deformed-to-Deformed Correspondences/BlueToy02/"+"deformed_"+str(num)+"_correspondences_icp.npy")

c=np.load("Deformed-to-Non-deformed Correspondences/BlueToy02/"+"non_deformed_"+str(num)+"_correspondences_icp.npy")
d=np.load("Non-deformed-to-Deformed Correspondences/BlueToy02/"+"non_deformed_"+str(num)+"_correspondences_icp.npy")

deformed_num_icp_deformed_to_non_yellow=a.tolist()
deformed_num_icp_non_to_deformed_yellow=b.tolist()
nondeformed_num_icp_deformed_to_non_yellow=c.tolist()
nondeformed_num_icp_non_to_deformed_yellow=d.tolist()
list_bijective_deformed_icp_yellow=[]
list_bijective_nondeformed_icp_yellow=[]

for i,row in enumerate(deformed_num_icp_deformed_to_non_yellow):
    try:
        id=deformed_num_icp_non_to_deformed_yellow.index(row)
        if nondeformed_num_icp_deformed_to_non_yellow[i]==nondeformed_num_icp_non_to_deformed_yellow[id]:
            list_bijective_deformed_icp_yellow.append(row)
            list_bijective_nondeformed_icp_yellow.append(nondeformed_num_icp_deformed_to_non_yellow[i])
    except:
        continue
print(len(list_bijective_deformed_icp_yellow),len(list_bijective_nondeformed_icp_yellow))
np.save("Bijective Correspondences/BlueToy02/deformed_1-"+str(num)+"_bijective_icp.npy",np.array(list_bijective_deformed_icp_yellow))
"""for row in nondeformed_num_icp_deformed_to_non_blue:
    if row in nondeformed_num_icp_non_to_deformed_blue:
        list_bijective_nondeformed_icp_blue.append(row)"""
np.save("Bijective Correspondences/BlueToy02/nondeformed_1-"+str(num)+"_bijective_icp.npy",np.array(list_bijective_nondeformed_icp_yellow))

e=np.load("Deformed-to-Non-deformed Correspondences/BlueToy02/"+"non_deformed_"+str(num)+"_correspondences_zoomed.npy")
f=np.load("Non-deformed-to-Deformed Correspondences/BlueToy02/"+"non_deformed_"+str(num)+"_correspondences_zoomed.npy")

g=np.load("Deformed-to-Non-deformed Correspondences/BlueToy02/"+"non_deformed_"+str(num)+"_correspondences_zoomed.npy")
h=np.load("Non-deformed-to-Deformed Correspondences/BlueToy02/"+"non_deformed_"+str(num)+"_correspondences_zoomed.npy")

deformed_num_zoomed_deformed_to_non_yellow=a.tolist()
deformed_num_zoomed_non_to_deformed_yellow=b.tolist()
nondeformed_num_zoomed_deformed_to_non_yellow=c.tolist()
nondeformed_num_zoomed_non_to_deformed_yellow=d.tolist()
list_bijective_deformed_zoomed_yellow=[]
list_bijective_nondeformed_zoomed_yellow=[]

for i,row in enumerate(deformed_num_zoomed_deformed_to_non_yellow):
    try:
        id=deformed_num_zoomed_non_to_deformed_yellow.index(row)
        if nondeformed_num_zoomed_deformed_to_non_yellow[i]==nondeformed_num_zoomed_non_to_deformed_yellow[id]:
            list_bijective_deformed_zoomed_yellow.append(row)
            list_bijective_nondeformed_zoomed_yellow.append(nondeformed_num_zoomed_deformed_to_non_yellow[i])
            #print(id,i,row,nondeformed_num_zoomed_deformed_to_non_blue[i],nondeformed_num_zoomed_non_to_deformed_blue[id])
    except:
        continue
print(len(list_bijective_deformed_zoomed_yellow),len(list_bijective_nondeformed_zoomed_yellow))
np.save("Bijective Correspondences/BlueToy02/deformed_1-"+str(num)+"_bijective_zoomed.npy",np.array(list_bijective_deformed_zoomed_yellow))
"""for row in nondeformed_num_zoomed_deformed_to_non_blue:
    if row in nondeformed_num_zoomed_non_to_deformed_blue:
        list_bijective_nondeformed_zoomed_blue.append(row)"""
np.save("Bijective Correspondences/BlueToy02/nondeformed_1-"+str(num)+"_bijective_zoomed.npy",np.array(list_bijective_nondeformed_zoomed_yellow))


110 110
110 110


FileNotFoundError: [Errno 2] No such file or directory: 'Bijective Correspondences/BlueToy02/YellowToy01/nondeformed_1-4_bijective_zoomed.npy'

In [4]:
np.save("Bijective Correspondences/BlueToy02/nondeformed_1-"+str(num)+"_bijective_zoomed.npy",np.array(list_bijective_nondeformed_zoomed_yellow))


In [5]:
k=np.load("Bijective Correspondences/BlueToy02/deformed_1-4_bijective_icp.npy")
k.shape

(110, 3)